In [ ]:
#Import necessary Python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#Load the data
quality = pd.read_csv("quality.csv")
quality_data = np.array(quality)

In [ ]:
#Split the training-testing sets
X = quality_data[:,1:-1]
y = quality_data[:,-1]
y = y.astype('int')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state= 88)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
model1 = linear_model.LogisticRegression()
Xlr_train = np.hstack((X_train[:,3].reshape(98, 1),X_train[:,4].reshape(98, 1)))
Xlr_test = np.hstack((X_test[:,3].reshape(33, 1),X_test[:,4].reshape(33, 1)))

model1.fit(Xlr_train,y_train.astype('int'))
print('Coefficient: ', model1.coef_)
print('Intercept: ', model1.intercept_)

ylr_pred = model1.predict(Xlr_test)
ylr_pred_prob =  model1.predict_proba(Xlr_test)

In [ ]:
#Load confusion matrix
from sklearn.metrics import confusion_matrix

CM_lr = confusion_matrix(y_test, ylr_pred)
print(CM_lr)

In [ ]:
#Calculate out-of-sample AUC
from sklearn.metrics import roc_curve,auc
fpr, tpr, thresholds = roc_curve(y_test.astype('int'), ylr_pred_prob[:,1])
#Draw the ROC curve
plt.plot(fpr,tpr,marker = 'o')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()
auc(fpr, tpr)

In [ ]:
#Normalize the OfficeVisits and Narcotics as covariates in KNN
OfficeVisits_train = ((X_train[:,3] - np.mean(X_train[:,3]))/np.std(X_train[:,3])).reshape(-1,1)
Narcotics_train = ((X_train[:,3] - np.mean(X_train[:,3]))/np.std(X_train[:,3])).reshape(-1,1)
OfficeVisits_test = ((X_test[:,3] - np.mean(X_test[:,3]))/np.std(X_test[:,3])).reshape(-1,1)
Narcotics_test = ((X_test[:,3] - np.mean(X_test[:,3]))/np.std(X_test[:,3])).reshape(-1,1)


In [ ]:
#Build the covariates of training and testing sets
X1_train = np.hstack((OfficeVisits_train,Narcotics_train))
X1_test = np.hstack((OfficeVisits_test,Narcotics_test))

In [ ]:
print(X1_train.shape)
print(X1_test.shape)

In [ ]:
#### Train the knn model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3,metric='euclidean')
knn.fit(X1_train, y_train)

In [ ]:
#Calculate the overall accuracy of knn model
OA_knn = knn.score(X1_test, y_test) 
print(OA_knn)

In [ ]:
#Confusion Matrix
y_pred = knn.predict(X1_test)

CM_knn = confusion_matrix(y_test, y_pred)
print(CM_knn)

In [ ]:
##Train-Validate-Test Triplet for the healthcare quality case
X1_trainval = X1_train
y_trainval = y_train

#Split the training and validation set
X1_train, X1_val, y_train, y_val = train_test_split(X1_trainval, y_trainval,test_size=0.33)

#Find out which model has the highest validation accuracy
val_scores = []
neighbors = np.arange(1, 15, 2)
for i in neighbors:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X1_train, y_train)
    val_scores.append(knn.score(X1_val, y_val))
    
print("Best validation accuracy: {:.3f}".format(np.max(val_scores)))
best_n_neighbors = neighbors[np.argmax(val_scores)]
print("Best number of neighbors: {}".format(best_n_neighbors))
knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)

#Test the performance of the "best" on the testing set 
knn.fit(X1_trainval, y_trainval)
print("Test-set accuracy: {:.3f}".format(knn.score(X1_test, y_test)))

In [ ]:
##k-fold Cross Validation for healthcare quality case

#import the CV package
from sklearn.model_selection import cross_val_score

#Cross Validation with number of folds k=3
cross_val_scores = []
for i in neighbors:
    knn = KNeighborsClassifier(n_neighbors=i)
    scores = cross_val_score(knn, X1_trainval, y_trainval, cv=6)
    cross_val_scores.append(np.mean(scores))
    
    
print("Best cross-validation accuracy: {:.3f}".format(np.max(cross_val_scores)))
best_n_neighbors = neighbors[np.argmax(cross_val_scores)]
print("Best number of neighbors: {}".format(best_n_neighbors))
knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
knn.fit(X1_trainval, y_trainval)
print("Test-set score: {:.3f}".format(knn.score(X1_test, y_test)))

In [ ]:
y_test

In [ ]:
##Lasso and Ridge Regression

#Load the boston housing data set
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
Xb = boston.data 

In [ ]:
yb = boston.target

In [ ]:
lasso_b = linear_model.Lasso(alpha=1)

In [ ]:
lasso_b.fit(Xb,yb)

In [ ]:
print(lasso_b.coef_)

In [ ]:
ridge_b = linear_model.Ridge(alpha=1)
ridge_b.fit(Xb,yb)
print(ridge_b.coef_)

In [ ]:
linear_b = linear_model.LinearRegression()
linear_b.fit(Xb,yb)
print(linear_b.coef_)

In [ ]:
#Tuning the parameter alpha for lasso and ridge regressions

#Import necessary package
from sklearn.model_selection import GridSearchCV


param_grid = {'alpha': np.logspace(-3, 0, 13)}
print(param_grid)

In [ ]:
grid_lasso = GridSearchCV(linear_model.Lasso(), param_grid, cv=10)
grid_lasso.fit(Xb, yb)
print(grid_lasso.best_params_)
print('Best R squared in Lasso Regression: %.2f' %grid_lasso.best_score_)

In [ ]:
grid_ridge = GridSearchCV(linear_model.Ridge(), param_grid, cv=10)
grid_ridge.fit(Xb, yb)
print(grid_ridge.best_params_)
print('Best R squared in Ridge Regression: %.2f' %grid_ridge.best_score_)